In [2]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

import pandas as pd

import IPython.terminal

from IPython.display import clear_output

# Titanic Dataset

In [3]:
X_train=pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
X_test=pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') #testing data

y_train=X_train.pop('survived')
y_test=X_test.pop('survived')

CATEGORIAL_COLUMNS=['sex', 'n_siblings_spouses', 'parch',  'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS=['age', 'fare']

feature_columns=[]
for feature_name in CATEGORIAL_COLUMNS:
    vocabulary=X_train[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds=tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds=ds.shuffle(1000)
        ds=ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn=make_input_fn(X_train, y_train)
test_input_fn=make_input_fn(X_test, y_test, num_epochs=1, shuffle=False)

linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn) #train
result=linear_est.evaluate(test_input_fn) # get model metrics/stats
clear_output()
print(result['accuracy'])
print(result)

result=list(linear_est.predict(test_input_fn))
for i in range(0,6):
    print(X_test.iloc[i])
    print(y_test.iloc[i])
    print(result[i]['probabilities'][0])
    print()

0.7537879
{'accuracy': 0.7537879, 'accuracy_baseline': 0.625, 'auc': 0.8353229, 'auc_precision_recall': 0.786507, 'average_loss': 0.47296044, 'label/mean': 0.375, 'loss': 0.4648334, 'precision': 0.6770833, 'prediction/mean': 0.36536378, 'recall': 0.65656567, 'global_step': 200}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\venka\AppData\Local\Temp\tmp39f1opw6\model.ckpt-200
INFO:tensorflow:

# Iris Dataset

In [7]:
CSV_COLUMN_NAMES=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES=['Setosa', 'Versicolor', 'Virginica']

train_path=tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path=tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train=pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test=pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train.head()

train_y=train.iloc[-1]
train.head()

def input_fn(features, label, training=True, batch_size=256):
    dataset=tf.data.Dataset.from_tensor_slices((dict(features), label))

    if training:
        dataset=dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

my_feature_columns=[]
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30, 10], n_classes=3)